# This notebook generate artificial meteor head-echo 

The meteor head echo is based on random meteor trajectory generated between bounds. The purpose of this notebook is testing of meteor estimator algorithm. 

In [3]:
% Observers coordinates in degrees and kilometers above sea level
global Obs = [
 5.51 47.34 0.28 % TX Coordinates (Graves)
 13.8986 49.8736 0.50 % RX ZEBRAK
 15.414812 50.409407 0.23 % RX NACHODSKO
 14.177880 50.229984 0.5 % RX Zvoleneves
 16.0115 50.5067 0.5 % RX OBSUPICE
 15.876004 49.207180 0.5 % RX DDMTREBIC
];

% Solver Boundaries: Lon   Lat   ASL    Ra  Dec   V
global LowerBound = [2.5; 41.60; 100; -360; -90; 12];
global UpperBound = [15.5; 50.34; 120;  720;  90; 72];

In [11]:
pkg load all
warning ("off");
format short g;
global Datetime Doppler tstart Obs_lla;
global GST = 0

%
% Degrees to radians
%
function r = deg2rad(d)
    r = d * pi / 180;
endfunction;

%
% Radians to degrees
%
function d = rad2deg(r)
    d = r * 180 / pi;
endfunction;

%
% Converts [lon lat asl] to [x y z]
%
function xyz = lla2xyz(lla)
    lon = lla(1);
    lat = lla(2);
    alt = lla(3) * 1000;
    a = 6378137;
    ee = 8.1819190842622e-2;
    asq = a * a;
    esq = ee * ee;
    N = a / sqrt(1 - esq * sin(lat) ^ 2);
    x = (N + alt) * cos(lat) * cos(lon) / 1000;
    y = (N + alt) * cos(lat) * sin(lon) / 1000;
    z = ((1 - esq) * N + alt) * sin(lat) / 1000;
    xyz = [x y z];
endfunction;

%
% Converts [x y z] to [lon lat asl]
%
function lla = xyz2lla(xyz)
    x = xyz(1) * 1000;
    y = xyz(2) * 1000;
    z = xyz(3) * 1000;
    a = 6378137;
    ee = 8.1819190842622e-2;
    asq = a * a;
    esq = ee * ee;
    b = sqrt(asq * (1 - esq));
    bsq = b * b;
    ep = sqrt((asq - bsq) / bsq);
    p = sqrt(x * x + y * y);
    th = atan2(a * z, b * p);
    lon = atan2(y, x);
    lat = atan2(z + (ep ^ 2) * b * (sin(th) ^ 3), (p - esq * a * (cos(th) ^ 3)));
    N = a / (sqrt(1 - esq * sin(lat) ^ 2));
    alt = (p / cos(lat) - N) / 1000;
    lla = [lon lat alt];
endfunction;

%
% Converts [Ra Dec Vel] velocity vector to [x y z] vector
%
function xyz = rdv2xyz(rdv)
    ra = rdv(1);
    dec = rdv(2);
    vel = rdv(3);
    x = - vel * cos(dec) * cos(ra);
    y = - vel * cos(dec) * sin(ra);
    z = - vel * sin(dec);
    xyz = [x y z];
endfunction;

%
% Converts [x y z] velocity vector to [Ra Dec Vel] vector
%
function rdv = xyz2rdv(xyz)
    global GST;
    v = norm(xyz);
    ra = GST + atan2(- xyz(2), - xyz(1));
    while (ra >= 2 * pi)
        ra = ra - 2 * pi;
    endwhile;
    while (ra < 0)
        ra = ra + 2 * pi;
    endwhile;
    dec = asin(- xyz(3) / norm(xyz));
    rdv = [ra dec v];
endfunction;

%
% Converts [Ra Dec Vel] into [Az El] from TX site
%
function ae = rdv2txae(rdv)
    global GST Obs_lla;
    txlon = Obs_lla(1, 1); % TX Site
    txlat = Obs_lla(1, 2);
    ra = rdv(1);
    dec = rdv(2);
    ha = GST + txlon - ra;
    sinalt = sin(dec) * sin(txlat) + cos(dec) * cos(txlat) * cos(ha);
    alt = asin(sinalt);
    cosaz = (sin(dec) - sin(alt) * sin(txlat)) / (cos(alt) * cos(txlat));
    az = acos(cosaz);
    if (sin(ha) >= 0)
        az = (2 * pi) - az;
    endif;
    ae = [az alt];
endfunction;

%
% Get sidereal time at Greenwich
%
function st = GreenwichSiderealTime(datetimestring)
    yr = str2num(datetimestring(1:4));
    mn = str2num(datetimestring(5:6));
    dy = str2num(datetimestring(7:8));
    hh = str2num(datetimestring(9:10));
    mm = str2num(datetimestring(11:12));
    ss = str2num(datetimestring(13:14));
    if (mn > 2)
        y = yr;
        m = mn;
    else
        y = yr - 1;
        m = mn + 12;
    endif;
    A = fix(y / 100);
    B = 2 - A + fix(A / 4);
    jd = fix(365.25 * y) + fix(30.6001 * (m + 1)) + dy + 1720994.5 + B;
    T = (jd - 2415020) / 36525;
    st = (0.276919398 + 100.0021359 * T + 0.000001075 * T * T);
    st = st + (((hh / 24) + (mm / 1440) + (ss / 86400)) * 1.002737908);
    st = deg2rad(360 * (st - fix(st)));
endfunction;

%
% Get angle between vectors
%
function angle = getAngle(a, b)
    angle = atan2(norm(cross(a, b)), dot(a, b));
endfunction;

%
% Get radiant elevation of a meteor (as seen from meteor itself)
%
function a = getMeteorElevation(P, V)
    a = getAngle(P, V) - (pi / 2);
endfunction;

%
% Get elevation of object (Obj) as seen from observer (Obs)
%
function a = getHorizonElevation(Obj, Obs)
    a = (pi / 2) - getAngle(Obs, Obj - Obs);
endfunction;

%
% Angular distance in the sky
%
function r = getAngularDistance(rdv1, rdv2)
    a1 = rdv1(1);
    d1 = rdv1(2);
    a2 = rdv2(1);
    d2 = rdv2(2);
    d = acos(sin(d1) * sin(d2) + cos(d1) * cos(d2) * cos(a1 - a2));
    v = abs(rdv1(3) - rdv2(3));
    r = [d v];
endfunction;

%
% Meteor Doppler function
%
function doppler = getDoppler(m, mv, tx, rx)
    mtx = tx - m;
    mrx = rx - m;
    vtx = dot(mv, mtx) / norm(mtx);
    vrx = dot(mv, mrx) / norm(mrx);
    doppler = (vtx + vrx) * 143050000 / 299792.458;
endfunction

%
% Solver function for fsolve
%
function M = meteorSolver(vector)
    global Obs Doppler;
    vector = vector';
    m = lla2xyz(vector(1:3));
    mv = rdv2xyz(vector(4:6));
    M = Doppler(:, 3);
    for n = 1:rows(M)
        r = Doppler(n, 1);
        time = Doppler(n, 2);
        M(n) = getDoppler(m + mv * time, mv, Obs(1, :), Obs(1 + r, :)) - M(n);
    endfor;
    M = M';
endfunction;


In [12]:
%
% Create a random meteor as starting point to the solver
%
function meteor = CreateRandomMeteor
    global LowerBound UpperBound;
    lon = LowerBound(1) + rand * (UpperBound(1) - LowerBound(1));
    lat = LowerBound(2) + rand * (UpperBound(2) - LowerBound(2));
    asl = LowerBound(3) + rand * (UpperBound(3) - LowerBound(3));
    %ra=LowerBound(4)+rand*(UpperBound(4)-LowerBound(4));
    ra = rand * 2 * pi;
    dec = LowerBound(5) + rand * (UpperBound(5) - LowerBound(5));
    v = LowerBound(6) + rand * (UpperBound(6) - LowerBound(6));
    meteor = [lon lat asl ra dec v];
endfunction;

%
% Check if a meteor is visible (and possible)
%
function vis = isVisible(meteor)
    global Obs;
    vis = false;
    P = lla2xyz(meteor(1:3));
    V = rdv2xyz(meteor(4:6));
    if getMeteorElevation(P, V) >= 0 % Falling meteor
        for n = 2:rows(Obs)
            %if getHorizonElevation(P,Obs(1,:)) >= 0        % Elevation from TX
            if getHorizonElevation(P, Obs(n, :)) >= 0 % Elevation from RXx
                vis = true;
            else
                break;
            endif;
        %endif;
        endfor;
    endif;
endfunction;

In [119]:
meteor

meteor =

      3.1057      47.399      111.08      4.9858     -58.773      43.368



In [105]:
4644.0   202.8  4497.2    7.72    2.78   10.20    2.50  44.24 100.00   97.5  -51.2  13.08  242.2  -79.9     0.65255 

parse error:

  syntax error

>>> 4644.0   202.8  4497.2    7.72    2.78   10.20    2.50  44.24 100.00   97.5  -51.2  13.08  242.2  -79.9     0.65255
                 ^



In [112]:
visibility = 0;
while visibility == 0
    meteor = CreateRandomMeteor;
    visibility =isVisible(meteor);
endwhile;
m = lla2xyz(meteor(1:3));
mv = rdv2xyz(meteor(4:6));

for site = 2:rows(Obs) 
    doppler_value = +inf;
    n=0;
    while doppler_value >= -50
        doppler_value = getDoppler(m + mv * time_step * n, mv, Obs(1, :), Obs(site, :));
        if doppler_value <= 100
            printf('%d %f %f\n', site - 1, time_step * n, doppler_value);
        endif;
        n = n + 1;
        if time_step * n >= 10 %check a total lenght of meteor flyby.
            break
        endif;
    endwhile;
    printf('\n');
endfor;

1 4.380000 96.867084
1 4.400000 91.324936
1 4.420000 85.782782
1 4.440000 80.240624
1 4.460000 74.698462
1 4.480000 69.156296
1 4.500000 63.614126
1 4.520000 58.071952
1 4.540000 52.529776
1 4.560000 46.987596
1 4.580000 41.445415
1 4.600000 35.903230
1 4.620000 30.361044
1 4.640000 24.818857
1 4.660000 19.276668
1 4.680000 13.734478
1 4.700000 8.192287
1 4.720000 2.650096
1 4.740000 -2.892096
1 4.760000 -8.434287
1 4.780000 -13.976478
1 4.800000 -19.518668
1 4.820000 -25.060857
1 4.840000 -30.603044
1 4.860000 -36.145230
1 4.880000 -41.687414
1 4.900000 -47.229596
1 4.920000 -52.771776

2 4.380000 95.985854
2 4.400000 90.443062
2 4.420000 84.900265
2 4.440000 79.357464
2 4.460000 73.814658
2 4.480000 68.271848
2 4.500000 62.729034
2 4.520000 57.186218
2 4.540000 51.643398
2 4.560000 46.100575
2 4.580000 40.557750
2 4.600000 35.014922
2 4.620000 29.472093
2 4.640000 23.929262
2 4.660000 18.386430
2 4.680000 12.843597
2 4.700000 7.300763
2 4.720000 1.757929
2 4.740000 -3.784906
2 4.7600

In [87]:
help(fprintf)

error: Invalid call to fprintf.  Correct usage is:

 -- Built-in Function: fprintf (FID, TEMPLATE, ...)
 -- Built-in Function: fprintf (TEMPLATE, ...)
 -- Built-in Function: NUMBYTES = fprintf (...)

Additional help for built-in functions and operators is
available in the online version of the manual.  Use the command
'doc <topic>' to search the manual index.

Help and information about Octave is also available on the WWW
at http://www.octave.org and via the help@octave.org
mailing list.


ans =          6
